In [1]:
import pickle
import pandas as pd

In [2]:
file_name_train_250 = "../Human Experiments/Data/RL_trajectories_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "../Human Experiments/Data/Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "../Human Experiments/Data/RL_trajectories_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "../Human Experiments/Data/Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [3]:
def create_train(trajectories_train):
    data_train = []
    for idxt, trajectory in enumerate(trajectories_train):
        min_price_index = trajectory.index(min(trajectory))
        trajectory_number = idxt
        for idxd, day in enumerate(trajectory):
            if(idxd <= min_price_index): 

                price = day
                if(idxd == min_price_index):
                    label = 1
                else:
                    label = 0
                data_train.append((trajectory_number, idxd, price, label))
            else:
                break
    return data_train

In [4]:
def create_test(trajectories_test):
    data_test = []
    for idxt, trajectory in enumerate(trajectories_test):
        min_price_index = trajectory.index(min(trajectory))
        trajectory_number = idxt
        for idxd, day in enumerate(trajectory):
            price = day
            if(idxd == min_price_index):
                label = 1
            else:
                label = 0
            data_test.append((trajectory_number, idxd, price, label))
    return data_test

In [5]:
data_train_250 = create_train(trajectories_train_250)
data_test_250 = create_test(trajectories_test_250)
data_train_500 = create_train(trajectories_train_500)
data_test_500 = create_test(trajectories_test_500)

In [6]:
dataframe_train_250 = pd.DataFrame(data_train_250, columns = ['trajectory_number','my_index' , 'price', 'label'])
dataframe_test_250 = pd.DataFrame(data_test_250, columns = ['trajectory_number','my_index' , 'price', 'label'])

dataframe_train_500 = pd.DataFrame(data_train_500, columns = ['trajectory_number','my_index' , 'price', 'label'])
dataframe_test_500 = pd.DataFrame(data_test_500, columns = ['trajectory_number','my_index' , 'price', 'label'])

In [7]:
Y_train_250 = dataframe_train_250.label
Y_test_250 = dataframe_test_250.label

Y_train_500 = dataframe_train_500.label
Y_test_500 = dataframe_test_500.label

In [8]:
X_train_250 = dataframe_train_250.drop("label", axis=1)
X_test_250 = dataframe_test_250.drop("label", axis=1)

X_train_500 = dataframe_train_500.drop("label", axis=1)
X_test_500 = dataframe_test_500.drop("label", axis=1)

## Random Forest Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
classifier_250 = RandomForestClassifier(n_estimators=100).fit(X_train_250,Y_train_250)
print("Accuracy 250: ", classifier_250.score(X_test_250, Y_test_250))

classifier_500 = RandomForestClassifier(n_estimators=100).fit(X_train_500,Y_train_500)
print("Accuracy 500: ", classifier_500.score(X_test_500, Y_test_500))


Accuracy 250:  0.8355555555555556
Accuracy 500:  0.8644444444444445


In [11]:
predictions_250 = list(classifier_250.predict(X_test_250))
X_test_250['predictions'] = predictions_250
X_test_250['true_labels'] = Y_test_250

predictions_500 = list(classifier_500.predict(X_test_500))
X_test_500['predictions'] = predictions_500
X_test_500['true_labels'] = Y_test_500

In [12]:
def get_final_output(dataframe):
    final_output = []
    for trajectory in range(test_size):
        current = dataframe.loc[(dataframe['trajectory_number'] == trajectory) & (dataframe['predictions'] == 1)]
        if(len(current.index) == 0):
            picked = dataframe.loc[(dataframe['trajectory_number'] == trajectory) & (dataframe['index'] == 14)]
            picked_tuple = (picked.price.values[0], int(picked.my_index.values[0]))
        else:
            picked = current.iloc[0]
            picked_tuple = (picked.price, int(picked.my_index))
        final_output.append(picked_tuple)    
    return final_output

In [13]:
final_output_250 = get_final_output(X_test_250)
final_output_500 = get_final_output(X_test_500)
final_output = [final_output_250, final_output_500]

In [14]:
final_output

[[(276.0, 14),
  (246.0, 14),
  (229.0, 13),
  (249.0, 14),
  (213.0, 14),
  (186.0, 4),
  (265.0, 14),
  (194.0, 5),
  (232.0, 10),
  (277.0, 14),
  (187.0, 0),
  (282.0, 14),
  (276.0, 14),
  (199.0, 7),
  (215.0, 9),
  (226.0, 8),
  (226.0, 11),
  (191.0, 2),
  (287.0, 14),
  (204.0, 3),
  (219.0, 6),
  (216.0, 2),
  (257.0, 14),
  (210.0, 9),
  (185.0, 2),
  (233.0, 14),
  (224.0, 11),
  (223.0, 2),
  (223.0, 10),
  (287.0, 14),
  (192.0, 5),
  (226.0, 2),
  (226.0, 10),
  (226.0, 12),
  (250.0, 14),
  (219.0, 14),
  (269.0, 14),
  (220.0, 1),
  (203.0, 1),
  (279.0, 14),
  (233.0, 13),
  (209.0, 14),
  (239.0, 14),
  (209.0, 9),
  (215.0, 13),
  (264.0, 14),
  (274.0, 14),
  (204.0, 11),
  (211.0, 9),
  (186.0, 4),
  (184.0, 2),
  (211.0, 1),
  (186.0, 0),
  (229.0, 14),
  (230.0, 14),
  (192.0, 1),
  (165.0, 3),
  (258.0, 14),
  (237.0, 1),
  (188.0, 8)],
 [(377.0, 2),
  (445.0, 10),
  (476.0, 14),
  (404.0, 4),
  (417.0, 4),
  (478.0, 14),
  (415.0, 11),
  (422.0, 11),
  (411.0,

In [15]:
final_output_file_name = "../Human Experiments/Tests/random_forest"
final_file_object = open(final_output_file_name, 'wb')
pickle.dump(final_output, final_file_object)
final_file_object.close()